In [1]:
from dotenv import load_dotenv
import os
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [2]:
load_dotenv()

True

In [ ]:
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()

# **all-MiniLM-L6-v2** model

In [ ]:
# practice
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embedding_function)
query = "How many quantity of the wallet is left?"
docs = db.similarity_search(query)
print(docs[0].page_content)


In [ ]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

# LangChain Expression with Chroma DB CSV (RAG) - **OpenAIEmbeddings** model

In [3]:
embeddings_ = OpenAIEmbeddings()
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()
db = Chroma.from_documents(documents, embeddings_)

In [4]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

Name: HOT WATER BOTTLE I AM SO POORLY
Quantity: 8
UnitPrice: 4.65


In [5]:
retriever = db.as_retriever()

template = """
You are the best online shopping mall seller.
Answer the question based only on the following context and basically recommend the product with unit price:
{context}

Question: {question}
"""
template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


/Users/choesuna/miniforge3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
print(chain.invoke("I am thirsty so want to buy some drinks."))
print(chain.invoke("I need a hanger to hang."))
print(chain.invoke("Soon my friend's birthday."))

Based on the given context, you can buy the "VINTAGE BILLBOARD DRINK ME MUG" which has a quantity of 6 and a unit price of 1.06.
Based on the provided context, you can consider purchasing the following hangers:
1. CROCHET ROSE DES CLOTHES HANGER - Quantity: 2, Unit Price: $2.95 each
2. HOME SWEET HOME 3 PEG HANGER - Quantity: 2, Unit Price: $3.75 each
3. DOOR HANGER MUM + DADS ROOM - Quantity: 1, Unit Price: $1.45
4. HOME SWEET HOME 3 PEG HANGER - Quantity: 1, Unit Price: $3.75

You have a variety of hangers to choose from based on your preferences and needs.
Based on the context provided, you could consider buying birthday items such as a "BLUE HAPPY BIRTHDAY BUNTING," "ROBOT BIRTHDAY CARD," "ELEPHANT BIRTHDAY CARD," and "TOAST ITS - HAPPY BIRTHDAY" to celebrate your friend's upcoming birthday.


# Question list
- I am thirsty so want to buy some drinks.
- Is there any wallet?
- Soon my friend's birthday.